# HR Analytics: 

## Project Overview
This notebook analyzes HR data to predict employee attrition and identify key factors contributing to employee turnover. The analysis is structured into logical sections for better understanding and reproducibility.

**Dataset:** HR Employee Data (556 employees, 41 features)  
**Goal:** Predict employee attrition and provide actionable insights

## 1. Import Libraries and Setup

This section imports all necessary libraries for data analysis, visualization, and machine learning.

In [2]:
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, confusion_matrix

In [10]:
# Load the HR dataset
print("Loading HR dataset...")

# Load data
RAW_CSV = 'hr_dataset.csv'
df = pd.read_csv(RAW_CSV, low_memory=False)

print(f"\u2713 Dataset loaded successfully!")
print(f"\nDataset Shape: {df.shape}")
print(f"\nColumns ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nFirst 3 rows:")
display(df.head(3))

print(f"\nBasic info about dataset:")
df.info()

Loading HR dataset...
✓ Dataset loaded successfully!

Dataset Shape: (556, 41)

Columns (41):
 1. EmployeeID
 2. FirstName
 3. LastName
 4. Gender
 5. Age
 6. MaritalStatus
 7. EducationLevel
 8. Department
 9. JobRole
10. BusinessUnit
11. HireDate
12. YearsAtCompany
13. YearsInCurrentRole
14. ManagerID
15. ManagerSatisfaction
16. PerformanceRating
17. TrainingHoursLastYear
18. Certifications
19. WorkLifeBalance
20. JobSatisfaction
21. Overtime
22. RemoteWorkFrequency
23. MonthlyIncome
24. BonusPercentage
25. StockOptionLevel
26. PromotionLast3Years
27. LeavesTaken
28. AbsenteeismDays
29. HealthInsurance
30. CommuteDistance
31. City
32. State
33. Country
34. TravelFrequency
35. ProjectsHandled
36. TeamSize
37. Attrition
38. ExitReason
39. RehireEligibility
40. EngagementScore
41. LastPerformanceReview

First 3 rows:


,EmployeeID,FirstName,LastName,Gender,Age,MaritalStatus,EducationLevel,Department,JobRole,BusinessUnit,...,State,Country,TravelFrequency,ProjectsHandled,TeamSize,Attrition,ExitReason,RehireEligibility,EngagementScore,LastPerformanceReview
0,1,Ahana,Bawa,Femle,55,Marrid,Doctorate,Marketing,HR Manager,Field Sales,...,Kerala,India,Frequently,20,14,Yes,Better Pay,No,100,25-11-2023
1,2,Adira,Dora,F,55,Single,High School,Supply Chain,DevOps Engineer,Field Sales,...,Kerala,India,Frequently,7,ten,No,Career Growth,Yes,24,26-10-2022
2,3,Mehul,Wadhwa,M,53,Single,High School,Sales,Legal Associate,Corporate,...,Gujarat,India,Frequently,8,ten,No,Career Growth,Yes,96,27-06-2013



Basic info about dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   EmployeeID             556 non-null    int64  
 1   FirstName              556 non-null    object 
 2   LastName               556 non-null    object 
 3   Gender                 556 non-null    object 
 4   Age                    556 non-null    int64  
 5   MaritalStatus          556 non-null    object 
 6   EducationLevel         556 non-null    object 
 7   Department             556 non-null    object 
 8   JobRole                556 non-null    object 
 9   BusinessUnit           556 non-null    object 
 10  HireDate               556 non-null    object 
 11  YearsAtCompany         556 non-null    int64  
 12  YearsInCurrentRole     556 non-null    int64  
 13  ManagerID              295 non-null    float64
 14  ManagerSatisfaction    556 non-

 ## 2) Setup: helper functions for cleaning

In [39]:
import re


def standardize_str_cols(ddf):
    """Trim whitespace, collapse repeated spaces, and lower-case short categorical columns."""
    for c in ddf.select_dtypes(include=['object']).columns:
        ddf[c] = ddf[c].astype(str).str.strip()
        ddf[c] = ddf[c].replace({'': np.nan, 'nan': np.nan})
        ddf[c] = ddf[c].str.replace(r'\s+', ' ', regex=True)
        try:
            nunique = ddf[c].nunique(dropna=True)
            if 1 < nunique < 100:
                ddf[c] = ddf[c].str.lower()
        except Exception:
            pass
    return ddf




def convert_numeric_like(ddf, sample_n=200):
    """Try to convert object columns that look numeric after removing common symbols."""
    for c in ddf.columns:
        if ddf[c].dtype == 'object':
            sample = ddf[c].dropna().astype(str).head(sample_n)
            if sample.empty:
                continue
            cleaned = sample.str.replace(r'[,₹$€¥\s]', '', regex=True)
            numeric_like = cleaned.str.match(r'^[+-]?\d+(\.\d+)?$').mean()
            if numeric_like > 0.8:
                ddf[c] = pd.to_numeric(ddf[c].astype(str).str.replace(r'[,₹$€¥\s]', '', regex=True), errors='coerce')
                print(f"Converted {c} -> numeric")
    return ddf




def parse_date_like(ddf):
    for c in ddf.columns:
        if ddf[c].dtype == 'object':
            sample = ddf[c].dropna().astype(str).head(50)
            if sample.empty:
                continue
            if sample.str.match(r'^\d{4}-\d{1,2}-\d{1,2}$').any() or sample.str.match(r'^\d{1,2}/\d{1,2}/\d{2,4}$').any():
                try:
                    ddf[c] = pd.to_datetime(ddf[c], infer_datetime_format=True, errors='coerce')
                    print(f"Parsed {c} -> datetime")
                except Exception:
                    pass
    return ddf

## 3) Cleaning pipeline

In [44]:
df = standardize_str_cols(df)

dup_count = df.duplicated(keep='first').sum()
print('Exact duplicate rows:', dup_count)
if dup_count > 0:
    df = df.drop_duplicates(keep='first')

df = convert_numeric_like(df)

df = parse_date_like(df)

missing_pct = df.isnull().mean().sort_values(ascending=False)
print('\nTop missing percentages:')
print(missing_pct.head(15))

high_missing = missing_pct[missing_pct > 0.5].index.tolist()
if high_missing:
    print('\nColumns with >50% missing (consider dropping or sourcing from HR):')
    for c in high_missing:
        print(' -', c)

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print('\nNumeric columns detected:', len(num_cols))
for c in num_cols:
    if df[c].dropna().shape[0] < 10:
        continue
    q01 = df[c].quantile(0.01)
    q99 = df[c].quantile(0.99)
    if q01 == q99:
        continue
    if df[c].skew(skipna=True) > 1.0 or df[c].skew(skipna=True) < -1.0:
        df[c] = np.where(df[c] < q01, q01, df[c])
        df[c] = np.where(df[c] > q99, q99, df[c])
        print(f"Winsorized {c}")

Exact duplicate rows: 0
Converted MonthlyIncome -> numeric

Top missing percentages:
LeavesTaken            0.492806
ManagerID              0.469424
ExitReason             0.289568
EmployeeID             0.000000
City                   0.000000
BonusPercentage        0.000000
StockOptionLevel       0.000000
PromotionLast3Years    0.000000
AbsenteeismDays        0.000000
HealthInsurance        0.000000
CommuteDistance        0.000000
State                  0.000000
RemoteWorkFrequency    0.000000
Country                0.000000
TravelFrequency        0.000000
dtype: float64

Numeric columns detected: 20
Winsorized TrainingHoursLastYear


## 4) Quick EDA: distributions and counts

In [49]:
num_summary = df[num_cols].describe().T
display(num_summary)
num_summary.to_csv(PLOTS_DIR / 'numeric_summary.csv')



,count,mean,std,min,25%,50%,75%,max
EmployeeID,556.0,278.500000,160.647648,1.00,139.75,278.5,417.25,556.0
Age,556.0,38.872302,13.733233,12.00,28.00,38.0,50.00,75.0
YearsAtCompany,556.0,8.239209,4.871166,0.00,4.00,9.0,12.00,16.0
YearsInCurrentRole,556.0,6.926259,4.804273,0.00,2.00,7.0,11.00,16.0
ManagerID,295.0,278.606780,159.189020,3.00,143.50,296.0,409.00,555.0
ManagerSatisfaction,556.0,3.028777,1.447918,1.00,2.00,3.0,4.00,5.0
PerformanceRating,556.0,3.625899,1.754370,1.00,2.00,4.0,5.00,6.0
TrainingHoursLastYear,556.0,50.541906,29.656306,0.55,24.00,50.0,76.00,99.0
Certifications,556.0,2.473022,1.699282,0.00,1.00,2.0,4.00,5.0
WorkLifeBalance,556.0,2.462230,1.734757,0.00,1.00,2.0,4.00,5.0


In [51]:
var_sorted = df[num_cols].var().sort_values(ascending=False)
top_nums = var_sorted.head(6).index.tolist()


for col in top_nums:
    plt.figure(figsize=(6,3))
    plt.hist(df[col].dropna(), bins=30)
    plt.title(f"Distribution: {col}")
    plt.xlabel(col)
    plt.ylabel('count')
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / f"hist_{col}.png")
    plt.close()


print('Saved histograms for top numeric features to', PLOTS_DIR)



Saved histograms for top numeric features to plots


In [53]:
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
for c in cat_cols[:10]:
    vc = df[c].value_counts(dropna=False).head(10)
    print('\nValue counts for', c)
    display(vc)

    plt.figure(figsize=(6,3))
    vc.plot(kind='bar')
    plt.title(f"Top values: {c}")
    plt.tight_layout()
    fn = PLOTS_DIR / f"cat_{c}.png"
    plt.savefig(fn)
    plt.close()


print('Saved categorical top-value barplots to', PLOTS_DIR)





Value counts for FirstName


FirstName
Manikya    8
Amani      8
Neysa      8
Gokul      7
Vanya      7
Piya       7
Faiyaz     7
Kanav      6
Yuvaan     6
Alia       6
Name: count, dtype: int64


Value counts for LastName


LastName
Dada      5
Bora      5
Buch      5
Barad     4
Shah      4
Bir       4
Karnik    4
Wagle     4
Bhat      4
Bumb      4
Name: count, dtype: int64


Value counts for Gender


...


Value counts for HireDate


HireDate
01-09-2014    2
08-11-2018    2
22-03-2022    2
29-08-2017    2
03-02-2023    2
21-10-2018    2
03-03-2024    2
07-05-2020    2
20-11-2020    2
18-11-2020    2
Name: count, dtype: int64


Value counts for Overtime


Overtime
no     287
yes    269
Name: count, dtype: int64

Saved categorical top-value barplots to plots


In [65]:
# Detect common attrition-like columns
possible_targets = ['attrition','left','is_left','resigned','separated','status','target']
detected_target = None
for c in df.columns:
    if c.lower() in possible_targets:
        detected_target = c
        break




In [67]:
if detected_target is None:
# fallback: find a 0/1 column
    for c in df.columns:
        vals = set(df[c].dropna().unique())
        if vals.issubset({0,1}):
            detected_target = c
            break


print('Detected target column:', detected_target)


Detected target column: Attrition


In [75]:
if detected_target is not None:
    print('\nTarget value counts:')
    display(df[detected_target].value_counts(dropna=False))

    if 'department' in [x.lower() for x in df.columns]:
        dept_col = [x for x in df.columns if x.lower() == 'department'][0]
        print("Department column detected, attempting to plot attrition by department...")
        print("Using department column:", dept_col)
        print("Target column:", detected_target, ", dtype:", df[detected_target].dtype)
        print("Unique values in target:", df[detected_target].dropna().unique())

        try:
        # ensure target is numeric (map yes/no etc.)
            target_vals = df[detected_target].copy()
            if target_vals.dtype == 'object' or str(target_vals.dtype).startswith('category'):
                mapping = {'yes': 1, 'no': 0, 'y': 1, 'n': 0,
                           'true': 1, 'false': 0,
                           'left': 1, 'stayed': 0,
                           'resigned': 1, 'active': 0, 'terminated': 1}
                mapped = target_vals.str.lower().map(mapping)
                if mapped.notnull().sum() > 0:
                    target_vals = mapped.fillna(target_vals)
            target_vals = pd.to_numeric(target_vals, errors='coerce')

            temp_df = pd.DataFrame({dept_col: df[dept_col], 'attrition_num': target_vals})

            plt.figure(figsize=(8, 3))
            df.groupby(dept_col)[target_vals].mean().sort_values(ascending=False).plot(kind='bar')
            plt.ylabel('Attrition rate')
            plt.title('Attrition rate by department')
            plt.tight_layout()
            plt.savefig(PLOTS_DIR / 'attrition_by_department.png')
            plt.close()
            print('Saved attrition_by_department.png')
        except Exception as e:
            print("Error while plotting department attrition:", e)


Target value counts:


Attrition
yes    283
no     273
Name: count, dtype: int64

Department column detected, attempting to plot attrition by department...
Using department column: Department
Target column: Attrition , dtype: object
Unique values in target: ['yes' 'no']
Error while plotting department attrition: 'Columns not found: 0, 1'


<Figure size 800x300 with 0 Axes>

## 5) Feature engineering

In [82]:
date_cols = df.select_dtypes(include=['datetime64[ns]']).columns.tolist()
if date_cols:
    ref_date = df[date_cols].max().max()
    for c in date_cols:
        df[f'tenure_days_from_{c}'] = (ref_date - df[c]).dt.days
        df[f'tenure_years_from_{c}'] = df[f'tenure_days_from_{c}'] / 365.25
    print('Derived tenure features from date columns:', date_cols)

In [84]:
for c in df.columns:
    if 'year' in c.lower() and pd.api.types.is_numeric_dtype(df[c]):
        try:
            df[c + '_bin'] = pd.cut(df[c], bins=[-0.1,1,3,5,10,100], labels=['0-1','1-3','3-5','5-10','10+'])
            print('Created bin for', c)
        except Exception:
            pass

Created bin for YearsAtCompany
Created bin for YearsInCurrentRole
Created bin for TrainingHoursLastYear
Created bin for PromotionLast3Years


In [90]:
manager_cols = [x for x in df.columns if 'manager' in x.lower()]
if manager_cols:
    mgr = manager_cols[0]

    if detected_target is not None and detected_target in df.columns:
        # Convert attrition target to numeric if categorical
        target_series = df[detected_target].copy()
        if target_series.dtype == 'object':
            target_series = target_series.str.strip().str.lower().map({'yes': 1, 'no': 0})
        elif target_series.dtype == 'bool':
            target_series = target_series.astype(int)

        df['_target_numeric'] = target_series

        agg = df.groupby(mgr)['_target_numeric'].mean().reset_index()
        agg = agg.rename(columns={'_target_numeric': 'manager_attrition_rate'})

        agg_file = PLOTS_DIR / 'manager_agg.csv'
        agg.to_csv(agg_file, index=False)
        print(' Saved manager-level attrition rate to', agg_file)
    else:
        print(" detected_target is None or not in df")


 Saved manager-level attrition rate to plots\manager_agg.csv


## 6) Save intermediate cleaned dataset

In [93]:
df.to_csv(CLEANED_CSV, index=False)
print('Saved cleaned dataset to', CLEANED_CSV)

Saved cleaned dataset to hr_dataset_cleaned.csv


## 7) Baseline predictive model

In [98]:
if detected_target is None:
    print('No attrition-like target detected — skipping modeling. If you want to model a specific column, set detected_target manually.')
else:
    y = df[detected_target].copy()

    if y.dtype == 'object' or y.dtype.name == 'category':
        mapping = {'yes':1,'no':0,'y':1,'n':0,'true':1,'false':0,'left':1,'stayed':0,'resigned':1,'active':0,'terminated':1}
        y_mapped = y.str.lower().map(mapping)
        if y_mapped.notnull().sum() > 0:
            y = y_mapped.fillna(y).astype(object)
            
    try:
        y_numeric = pd.to_numeric(y, errors='coerce')
        if y_numeric.notnull().sum() > 0 and y_numeric.nunique() <= 10:
            y = y_numeric
    except Exception:
        pass

    mask = y.notnull()
    X = df.loc[mask].drop(columns=[detected_target])
    y = y.loc[mask]
    print('Modeling dataset:', X.shape, '->', y.shape)

    miss_pct = X.isnull().mean()
    drop_cols = miss_pct[miss_pct > 0.6].index.tolist()
    if drop_cols:
        print('Dropping high-missing columns:', drop_cols)
        X = X.drop(columns=drop_cols)

    num_feats = X.select_dtypes(include=[np.number]).columns.tolist()
    cat_feats = X.select_dtypes(include=['object','category','datetime']).columns.tolist()

    cat_small = [c for c in cat_feats if X[c].nunique(dropna=True) < 50]
    cat_large = [c for c in cat_feats if X[c].nunique(dropna=True) >= 50]


    print('Numeric features:', len(num_feats), 'Small cat feats:', len(cat_small), 'Large cat feats:', len(cat_large))


    num_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])


    preproc = ColumnTransformer([
        ('num', num_pipe, num_feats),
        ('cat', cat_pipe, cat_small)
    ], remainder='drop')


    model = Pipeline([('preproc', preproc), ('clf', RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1))])
    # split
    strat = y if y.nunique() <= 2 else None
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=strat)


    print('Training...')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    print('\nClassification report (test set):')
    print(classification_report(y_test, y_pred))


# ROC AUC if binary
    try:
        if y.nunique() == 2:
            y_proba = model.predict_proba(X_test)[:,1]
            auc = roc_auc_score(pd.to_numeric(y_test), y_proba)
            print('ROC AUC:', round(auc,4))
    except Exception:
        pass


# cross-val accuracy
    try:
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) if y.nunique() <= 10 else 5
        scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
        print('CV accuracy: %.4f +/- %.4f' % (scores.mean(), scores.std()))
        pd.Series(scores).to_csv(PLOTS_DIR / 'cv_scores.csv', index=False)
    except Exception as e:
        print('Cross-val failed:', e)


# Feature importances
    try:
        ohe = model.named_steps['preproc'].named_transformers_['cat'].named_steps['ohe']
        ohe_cols = ohe.get_feature_names_out(cat_small).tolist() if len(cat_small)>0 else []
        feat_names = num_feats + ohe_cols
        importances = model.named_steps['clf'].feature_importances_
        fi = pd.DataFrame({'feature': feat_names, 'importance': importances}).sort_values('importance', ascending=False).head(30)
        display(fi.head(20))
        fi.to_csv(PLOTS_DIR / 'feature_importances_top30.csv', index=False)


# plot top 10
        top10 = fi.head(10).iloc[::-1]
        plt.figure(figsize=(8,4))
        plt.barh(top10['feature'], top10['importance'])
        plt.xlabel('Importance')
        plt.title('Top 10 feature importances')
        plt.tight_layout()
        plt.savefig(PLOTS_DIR / 'feature_importances_top10.png')
        plt.close()
        print('Saved feature importance plot to', PLOTS_DIR / 'feature_importances_top10.png')
    except Exception as e:
        print('Could not compute feature importances cleanly:', e)

Modeling dataset: (556, 45) -> (556,)
Numeric features: 21 Small cat feats: 20 Large cat feats: 4
Training...

Classification report (test set):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        55
           1       1.00      1.00      1.00        57

    accuracy                           1.00       112
   macro avg       1.00      1.00      1.00       112
weighted avg       1.00      1.00      1.00       112

ROC AUC: 1.0
CV accuracy: 1.0000 +/- 0.0000


,feature,importance
20,_target_numeric,0.437437
11,MonthlyIncome,0.023404
7,TrainingHoursLastYear,0.023104
0,EmployeeID,0.022825
1,Age,0.021340
19,EngagementScore,0.018663
17,CommuteDistance,0.017904
18,ProjectsHandled,0.016604
16,AbsenteeismDays,0.016317
12,BonusPercentage,0.015573


Saved feature importance plot to plots\feature_importances_top10.png


## 8) Insights

In [101]:
print('\nNotebook run finished. Generated plots and cleaned CSV in /mnt/data')
print('Cleaned CSV:', CLEANED_CSV)
print('Plots directory:', PLOTS_DIR)


Notebook run finished. Generated plots and cleaned CSV in /mnt/data
Cleaned CSV: hr_dataset_cleaned.csv
Plots directory: plots
